In [2]:
import os
import gensim
import spacy
from nltk.tokenize import PunktSentenceTokenizer
from collections import Counter

# None of this is neeeded as the inaugural library does it all already

def read_file(file_name):
  with open(file_name, 'r+', encoding='utf-8') as file:
    file_text = file.read()
  return file_text

def process_speeches(speeches):
  word_tokenized_speeches = list()
  for speech in speeches:
    sentence_tokenizer = PunktSentenceTokenizer()
    sentence_tokenized_speech = sentence_tokenizer.tokenize(speech)
    word_tokenized_sentences = list()
    for sentence in sentence_tokenized_speech:
      word_tokenized_sentence = [word.lower().strip('.').strip('?').strip('!') for word in sentence.replace(",","").replace("-"," ").replace(":","").split()]
      word_tokenized_sentences.append(word_tokenized_sentence)
    word_tokenized_speeches.append(word_tokenized_sentences)
  return word_tokenized_speeches

def merge_speeches(speeches):
  all_sentences = list()
  for speech in speeches:
    for sentence in speech:
      all_sentences.append(sentence)
  return all_sentences

def get_president_sentences(president):
  files = sorted([file for file in os.listdir() if president.lower() in file.lower()])
  speeches = [read_file(file) for file in files]
  processed_speeches = process_speeches(speeches)
  all_sentences = merge_speeches(processed_speeches)
  return all_sentences

def get_presidents_sentences(presidents):
  all_sentences = list()
  for president in presidents:
    files = sorted([file for file in os.listdir() if president.lower() in file.lower()])
    speeches = [read_file(file) for file in files]
    processed_speeches = process_speeches(speeches)
    all_prez_sentences = merge_speeches(processed_speeches)
    all_sentences.extend(all_prez_sentences)
  return all_sentences

def most_frequent_words(list_of_sentences):
  all_words = [word for sentence in list_of_sentences for word in sentence]
  return Counter(all_words).most_common()

In order to create word embeddings on the corpus of all the presidents’ speeches, we need to read the text data from each file, separate the files into sentences on a word by word basis, and then merge all the sentences across the speeches into one big list of lists.

Let’s start by finding all the file names for the .txt files we will be analyzing

In [3]:
files = sorted([file for file in os.listdir() if file[-4:] == '.txt'])

In [4]:
from nltk.corpus import inaugural
files = inaugural.fileids()

speeches = []

for file in files:
    line = inaugural.sents(file)
    speeches.append(line)

print(speeches[0][0][0])
print(speeches[0][0])
print(speeches[0])
# processed_speeches = process_speeches(speeches)

Fellow
['Fellow', '-', 'Citizens', 'of', 'the', 'Senate', 'and', 'of', 'the', 'House', 'of', 'Representatives', ':']
[['Fellow', '-', 'Citizens', 'of', 'the', 'Senate', 'and', 'of', 'the', 'House', 'of', 'Representatives', ':'], ['Among', 'the', 'vicissitudes', 'incident', 'to', 'life', 'no', 'event', 'could', 'have', 'filled', 'me', 'with', 'greater', 'anxieties', 'than', 'that', 'of', 'which', 'the', 'notification', 'was', 'transmitted', 'by', 'your', 'order', ',', 'and', 'received', 'on', 'the', '14th', 'day', 'of', 'the', 'present', 'month', '.'], ...]


In [5]:
# Create list of sentences for all presidents
sentences_all_presidents = [' '.join(speech) for speech in inaugural.sents()]

# Function to tokenize all sentences and remove punctuation, put in consistent case, etc
def tokenize_sentences(sentences_to_tokenize):
    word_tokenized_sentences = []
    for sentence in sentences_to_tokenize:
        for word in sentence:
            word_tokenized_sentence = [word.lower().strip('.').strip('?').strip('!') for word in sentence.replace(",","").replace("-"," ").replace(":","").split()]
            word_tokenized_sentences.append(word_tokenized_sentence)
    return word_tokenized_sentences

# Tokenize sentences from all presidents
tokenized_sentences_all_presidents = tokenize_sentences(sentences_all_presidents)




In [6]:
# Function to find the most frequent words
def most_freq_words(tokenized_sentences):
    sent_in_speeches = [word for sentence in tokenized_sentences for word in sentence]
    return Counter(sent_in_speeches).most_common()

# Find most frequent words for all presidents
most_freq_words_all_presidents = most_freq_words(tokenized_sentences_all_presidents)
print(most_freq_words_all_presidents)

[('the', 2916663), ('of', 2093312), ('and', 1625257), ('to', 1321574), ('', 806243), ('in', 748523), ('a', 632069), ('our', 488696), ('that', 488385), ('be', 408560), (';', 347103), ('by', 338805), ('it', 326658), ('for', 325734), ('which', 310041), ('we', 296858), ('as', 283540), ('is', 276996), ('with', 276548), ('all', 248646), ('have', 232691), ('their', 226032), ('i', 216813), ('not', 216398), ('will', 193127), ('this', 184759), ('or', 169190), ('people', 168477), ('government', 167838), ('its', 166288), ('are', 158856), ('on', 158015), ('my', 157372), ('but', 154421), ('from', 151715), ('if', 147417), ('has', 145630), ('an', 140900), ('been', 140191), ('so', 116639), ('upon', 113102), ('can', 110659), ('states', 109963), ('they', 105440), ('may', 102999), ('them', 99889), ('us', 98530), ('no', 97896), ('shall', 96146), ('who', 92546), ('every', 90531), ('at', 89528), ('country', 86770), ('public', 85613), ('those', 83350), ('great', 83000), ('more', 80011), ('should', 79858), ('p

In [7]:
# Create a word embedding model with gensim
# Train it on the sentences in speeches
# Iniitally was vector_size=96, window=5, min_count=1, workers=2, sg=1. Reduced to get it to go faster
all_prez_embeddings = gensim.models.Word2Vec(tokenized_sentences_all_presidents, vector_size=96, window=2, min_count=10, workers=2, sg=1)

# Find words most similar to 'freedom'
similar_to_freedom = all_prez_embeddings.wv.most_similar('freedom')
print(similar_to_freedom)

[('peace', 0.4726826548576355), ('insatiable', 0.4404040575027466), ('america', 0.4382684826850891), ('a', 0.4381425082683563), ('the', 0.43538758158683777), ('deadlock', 0.4222777783870697), ('it', 0.4220125675201416), ('de', 0.4215458333492279), ('beseech', 0.4203735291957855), ('inaction', 0.4194783568382263)]


In [8]:
# Find words most similar to 'spirit'
similar_to_spirit = all_prez_embeddings.wv.most_similar('spirit')
print(similar_to_spirit)

[('citizenship', 0.5116673707962036), ('retarded', 0.45807769894599915), ('disposing', 0.45553532242774963), ('pride', 0.4473133981227875), ('tapping', 0.43829137086868286), ('amen', 0.4242631494998932), ('trait', 0.4189799726009369), ('ennobling', 0.41644737124443054), ('proclaim', 0.4148845672607422), ('referendum', 0.4109589755535126)]


A fun aspect of word embeddings is to see how different corpora result in different word embeddings, alluding to differences in how words are used between writers/authors/speakers.

Let’s train a word embedding model on a single president and see how their word embeddings differ from the collection of all presidents.

In [9]:
# sentences_roosevelt = inaugural.sents(['1945-Roosevelt.txt', '1941-Roosevelt.txt', '1937-Roosevelt.txt', '1933-Roosevelt.txt'])

sentences_roosevelt = [' '.join(sublist) for sublist in inaugural.sents(['1945-Roosevelt.txt', '1941-Roosevelt.txt', '1937-Roosevelt.txt', '1933-Roosevelt.txt'])]

tokenized_sentences_roosevelt = tokenize_sentences(sentences_roosevelt)

most_freq_words_roosevelt = most_freq_words(tokenized_sentences_roosevelt)

roosevelt_embeddings = gensim.models.Word2Vec(tokenized_sentences_roosevelt, vector_size=96, window=5, min_count=1, workers=2, sg=1)

# Find words most similar to 'freedom'
similar_to_freedom = roosevelt_embeddings.wv.most_similar('freedom')
print(similar_to_freedom)

[('privilege', 0.7743335962295532), ('ebbing', 0.6940739154815674), ('tide', 0.6467085480690002), ('voices', 0.6065096855163574), ('wave', 0.5903862714767456), ('story', 0.5659064650535583), ('rises', 0.565047025680542), ('symptoms', 0.5535821914672852), ('surging', 0.5390881896018982), ('should', 0.5321263074874878)]
